<a href="https://colab.research.google.com/github/lqiang67/rectified-flow/blob/main/examples/interpolation_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/lqiang67/rectified-flow.git
%cd rectified-flow/

In [1]:
import torch
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import warnings
import copy
import plotly.graph_objects as go

import torch.distributions as dist

from rectified_flow.utils import set_seed
from rectified_flow.utils import match_dim_with_data
from rectified_flow.datasets.toy_gmm import TwoPointGMM

from rectified_flow.rectified_flow import RectifiedFlow
from rectified_flow.models.toy_mlp import MLPVelocityConditioned, MLPVelocity

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Interpolations

In this notebook, we will first demonstrate that all *affine interpolations* are point-wise transformable and explain how transformations between these interpolations can be performed. Then, we will show that these interpolations yield essentially **equivalent** rectified flow dynamics and rectified couplings. Surprisingly, transformations applied to the interpolation are **exactly the same as** those applied to the rectified flow.

You can check this [blog post](https://rectifiedflow.github.io/blog/2024/interpolation/) for more comprehensive discussion.

Let's start by reviewing the basic concepts of Rectified Flow. If you're looking for a more detailed explanation of the process, we recommend checking out the `train_2d_toy` notebook first.

**Interpolation**

Given observed samples $X_0 \sim \pi_0$ from source distribution and $X_1 \sim \pi_1$ from target distribution, we consider a class of *affine interpolations* $X_t$:

$$
X_t = \alpha_t \cdot X_0 + \beta_t \cdot X_1,
$$

where $ \alpha_t $ and $ \beta_t $ are time-dependent functions satisfying:
$$
\alpha_0 = \beta_1 = 0 \quad \text{and} \quad \alpha_1 = \beta_0 = 1.
$$

This interpolation scheme is referred to as **affine interpolation**. In practice, it is often desirable for $\alpha_t$ to be monotonically increasing and $\beta_t$ to be monotonically decreasing over time $[0,1]$.

We refer to $\{X_t\} = \{X_t : t \in [0,1]\}$ as the **interpolation process**, which smoothly transitions or "bridges" the distribution between $X_0$ and $X_1$.

While this process effectively creates a bridge between $X_0$ and $X_1$, it has a significant limitation: it is not "simulatable." Generating $X_t$ at an intermediate time $t$ requires access to both $X_0$ and $X_1$, rather than evolving solely from $X_0$. This dependency makes it impossible for generating new samples in $X_1$, as the target distribution $X_1$ must already be known.

**Rectified Flow Velocity Field**

To make the interpolation process "simulatable," we can train an Ordinary Differential Equation (ODE) model. The idea is to model the process with an ODE defined as $\dot{Z}_t = v_t(Z_t)$, where the velocity field $v_t$ is trained to match the slope $\dot{X}_t$ of the interpolation process. This can be achieved by minimizing the following objective:
$$
\min_v \int_0^1 \mathbb{E} \left[\left\| \dot{X}_t - v_t(X_t) \right\|^2 \right] \, \mathrm{d}t.
$$
The theoretical optimum is given by:
$$
v_t^*(x) = \mathbb{E}[\dot{X}_t \mid X_t = x],
$$

which represents the **conditional average** of all slopes $\dot{X}_t$ of the interpolation process at a specific point $X_t = x$.

A crucial implication of this conditional average is that it preserves the marginals of the distribution. Intuitively, the ODE model ensures that the total "mass" or "particles" passing through any small local area remains the same after rectifying. As a result, the distributions of $\{Z_t\}_t$ (simulated using the ODE) and $\{X_t\}_t$ (from the interpolation process) are guaranteed to be the same. This property is key to ensuring that the ODE correctly models the desired transformation between distributions.

![cross](https://github.com/lqiang67/rectified-flow/blob/main/assets/flow_in_out.png?raw=1)

We refer to the process $\{Z_t\}$ as the **rectified flow**, which is induced from the interpolation process $\{X_t\}$. The rectified flow follows the dynamics:

$$
Z_t = Z_0 + \int_0^t v(Z_t, t) \, \mathrm{d}t, \quad \forall t \in [0, 1], \quad Z_0 = X_0,
$$
or more compactly:
$$
\{Z_t\} = \texttt{RectFlow}(\{X_t\}).
$$
In the 2D toy example, we used the `straight` interpolation, where the interpolation coefficients are defined as $\alpha_t = 1 - t$ and $\beta_t = t$. This results in:

$$
X_t = tX_1 + (1 - t)X_0, \quad \dot{X}_t = X_1 - X_0.
$$
However, $\alpha_t$ and $\beta_t$ are not limited to this specific choice. They can be **any** time-dependent functions, as long as they satisfy the conditions $\alpha_0 = \beta_1 = 0$ and $ \alpha_1 = \beta_0 = 1$ (and also the monotonic property). This means there are infinitely many possible interpolation processes $\{X_t\}$ that can be used to induce rectified flows.

Let’s review three widely used interpolation schemes::

**Straight Line Interpolation** (`"straight"` or `"lerp"`)
   
$$
\begin{align}
    \alpha_t & = t,       & \beta_t & = 1 - t \\
    \dot{\alpha}_t & = 1, & \dot{\beta}_t & = -1
\end{align}
$$
- This interpolation follows a straight line connecting the source and target distributions with constant speed.

**Spherical Interpolation** (`"spherical"` or `"slerp"`)

$$
\begin{align}
    \alpha_t & = \sin\left(\frac{\pi}{2} t\right), & \beta_t & = \cos\left(\frac{\pi}{2} t\right) \\
    \dot{\alpha}_t & = \frac{\pi}{2} \cos\left(\frac{\pi}{2} t\right), & \dot{\beta}_t & = -\frac{\pi}{2} \sin\left(\frac{\pi}{2} t\right)
\end{align}
$$

- This slerp spherical interpolation forms a curved trajectory, note that in both cases the boundary conditions are satisfied.

**DDIM / VP ODE Interpolation**

$$
\begin{align}
	\alpha_t &= \exp \left(- \frac{1}{4}a(1-t)^2-\frac 1 2b(1-t) \right), \quad \beta_t = \sqrt{1- \alpha_t^2}
\end{align}
$$
- With default values: $a=19.9, b=0.1$
- This also forms a spherical curve, but with non-uniform speed defined by $\alpha_t$.

In [ ]:
from rectified_flow.datasets.toy_gmm import TwoPointGMM

set_seed(0)
n_samples = 50000
pi_0 = dist.MultivariateNormal(torch.zeros(2, device=device), torch.eye(2, device=device))
pi_1 = TwoPointGMM(x=15.0, y=2, std=0.3)
D0 = pi_0.sample([n_samples])
D1, labels = pi_1.sample_with_labels([n_samples])
labels.tolist()

from rectified_flow.flow_components.interpolation_solver import AffineInterp
from rectified_flow.utils import visualize_2d_trajectories_plotly

straight_interp = AffineInterp("straight")
spherical_interp = AffineInterp("spherical")

idx = torch.randperm(n_samples)[:1000]
x_0 = D0[idx]
x_1 = D1[idx]

print(x_0.shape)

straight_interp_list = []
spherical_interp_list = []

for t in np.linspace(0, 1, 50):
	x_t_straight, dot_x_t_straight = straight_interp.forward(x_0, x_1, t)
	x_t_spherical, dot_x_t_spherical = spherical_interp.forward(x_0, x_1, t)
	straight_interp_list.append(x_t_straight)
	spherical_interp_list.append(x_t_spherical)

visualize_2d_trajectories_plotly(
	trajectories_dict={"straight interp": straight_interp_list, "spherical interp": spherical_interp_list},
	D1_gt_samples=D1[:5000],
	num_trajectories=50,
	title="Interpolated Trajectories Visualization",
)

# Interpolation convertor

In this section, we'll demonstrate how to convert between two interpolation processes.

Consider two affine interpolation processes defined with same coupling $(X_0, X_1)$:
$$
X_t = \alpha_t X_1 + \beta_t X_0 \quad \text{and} \quad X_{t}' = \alpha_{t}' X_1 + \beta_{t}' X_0.
$$

We can easily convert between these two interpolations using the following two steps:

**1. Matching Time $t$**

To match the time parameter $t$ between the processes, note the following properties:
$$
\dot{\alpha}_t > 0, \quad \dot{\beta}_t < 0, \quad \alpha_t, \beta_t \in [0, 1], \quad \forall t \in [0, 1].
$$
These conditions imply that the ratio $ \alpha_t / \beta_t$ is **strictly increasing** over the interval $[0, 1]$. Consequently, for any given $t$ in process $\{X_t'\}$, there exists a unique $t'$ in process $\{X_t\}$ such that the ratio matches:
$$
\frac{\alpha_{t'}}{\beta_{t'}} = \frac{\alpha_t'}{\beta_t'}.
$$
Similarly, for any given $t'$ in $\{X_t\}$, there exists a unique $t$ in $\{X_t'\}$ such that the ratio matches. This establishes a **bijective mapping** between $t$ and $t'$.

**2. Matching Scales**

Once the times $t$ and $t'$ are matched, we consider the scale factors:
$$
\frac{X_{t'}}{X_t'} = \frac{\alpha_{t'}X_1 + \beta_{t'}X_0}{\alpha_t'X_1 + \beta_t'X_0}.
$$
Rewriting this ratio:
$$
\frac{X_{t'}}{X_t'} = \frac{\alpha_{t'}}{\alpha_t'} \cdot \frac{X_1 + \frac{\beta_{t'}}{\alpha_{t'}} X_0}{X_1 + \frac{\beta_t'}{\alpha_t'} X_0} = \frac{\alpha_{t'}}{\alpha_t'}.
$$
This implies the scaling factor:
$$
\omega_t := \frac{\alpha_{t'}}{\alpha_t'} = \frac{\beta_{t'}}{\beta_t'} = \frac{X_{t'}}{X_t'}.
$$
**Pointwise Transformability**

Formally, we define two interpolation processes $\{X_t\}$ and $\{X_t'\}$ to be **pointwise transformable** if:
$$
X_t' = \phi_t(X_{\tau_t}), \quad \forall t \in [0, 1],
$$
where:

- $\tau: t \mapsto \tau_t$ is a monotonic time transformation (also invertible).
- $\phi: (t, x) \mapsto \phi_t(x)$ is an invertible scaling transformation.

For the example above:

- $\tau_t = t'$ (time transformation),
- $\phi_t(X_t) = X_{\tau_t}/\omega_t$ (scaling transformation).

We can determine the time scaling function $\tau_t$ in two ways. For simple cases, $\tau_t$ can be computed analytically. For more complex scenarios, a numerical approach, such as a simple binary search, can be used to find $\tau_t$ efficiently. 

In [ ]:
from rectified_flow.flow_components.interpolation_convertor import AffineInterpConverter

straight_interp_list = []
spherical_interp_list = []
straight_to_spherical_interp_list = []
spherical_to_straight_interp_list = []

for t in np.linspace(0, 1, 50):
	t = match_dim_with_data(t, x_0.shape, x_0.device, x_0.dtype, expand_dim=False)

	x_t_straight, dot_x_t_straight = straight_interp.forward(x_0, x_1, t)
	x_t_spherical, dot_x_t_spherical = spherical_interp.forward(x_0, x_1, t)
	straight_interp_list.append(x_t_straight)
	spherical_interp_list.append(x_t_spherical)

	# convert straight_interp to spherical_interp
	matched_t_straight, scaling_factor_straight = AffineInterpConverter.match_time_and_scale(straight_interp, spherical_interp, t)
	# print(t[:5], matched_t_straight[:5], scaling_factor_straight[:5])
	x_t_straight_to_spherical, dot_x_t_straight_to_spherical = straight_interp.forward(x_0, x_1, matched_t_straight)
	x_t_straight_to_spherical = x_t_straight_to_spherical / scaling_factor_straight.unsqueeze(-1)
	straight_to_spherical_interp_list.append(x_t_straight_to_spherical)

	# convert spherical_interp to straight_interp
	matched_t_spherical, scaling_factor_spherical = AffineInterpConverter.match_time_and_scale(spherical_interp, straight_interp, t)
	x_t_spherical_to_straight, dot_x_t_spherical_to_straight = spherical_interp.forward(x_0, x_1, matched_t_spherical)
	x_t_spherical_to_straight = x_t_spherical_to_straight / scaling_factor_spherical.unsqueeze(-1)
	spherical_to_straight_interp_list.append(x_t_spherical_to_straight)

visualize_2d_trajectories_plotly(
	trajectories_dict={
		"straight interp": straight_interp_list,
		"spherical interp": spherical_interp_list,
		"straight to spherical interp": straight_to_spherical_interp_list,
		"spherical to straight interp": spherical_to_straight_interp_list,
	},
	D1_gt_samples=D1[:2000],
	num_trajectories=100,
	title="Interpolated Trajectories Visualization",
)

As shown in the figure above, the transformed trajectories match perfectly.

Interestingly, the same transformation applied to the interpolation process $ \{X_t\} $ can also be applied to the corresponding rectified flows. This leads to the following theorem:


### **Theorem**

If two processes $ \{X_t\} $ and $ \{X'_t\} $ are related pointwise by:

$$
X'_t = \phi_t(X_{\tau_t}),
$$

where $ \phi : (t, x) \mapsto \phi_t(x) $ and $ \tau : t \mapsto \tau_t $ are differentiable and invertible maps, then their corresponding rectified flows, $ \{Z_t\} $ and $ \{Z'_t\} $, satisfy the same relationship:

$$
Z'_t = \phi_t(Z_{\tau_t}),
$$

provided this relationship holds at initialization, i.e., $ Z'_0 = \phi_0(Z_0)$.


**Implications**

This result shows that the rectified flows of pointwise transformable interpolations are fundamentally the same, up to the same pointwise transformation. Furthermore, if $ X_t = \mathcal{I}_t(X_0, X_1) $ and $ X'_t = \mathcal{I}'_t(X_0, X_1) $ are constructed from the same coupling $ (X_0, X_1) $, they yield identical rectified couplings: $ (Z'_0, Z'_1) = (Z_0, Z_1) $.

To summarize, let $\{X'_t\} = \texttt{Transform}(\{X_t\}) $ represent the pointwise transformation described above. The result implies that the rectification operation $ \texttt{Rectify}(\cdot) $ is **equivariant** under these transformations, meaning:

$$
\texttt{Rectify}(\texttt{Transform}(\{X_t\})) = \texttt{Transform}(\texttt{Rectify}(\{X_t\})).
$$

See Chapter 3 of the flow book for detailed derivation.

This figure illustrates the transformation between two interpolations, specifically $ \tau_t $ and $ \omega_t $.

We can see that the only difference between the `ddim` and `spherical` is the time scaling.

In [ ]:
t = torch.linspace(0, 1, 100)
matched_t, scaling_factor = AffineInterpConverter.match_time_and_scale(AffineInterp("straight"), AffineInterp("spherical"), t)
# matched_t, scaling_factor = AffineInterpConverter.match_time_and_scale(AffineInterp("ddim"), AffineInterp("spherical"), t)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=matched_t, mode='lines', name='matched_t'))
fig.add_trace(go.Scatter(x=t, y=scaling_factor, mode='lines', name='scaling_factor'))
fig.update_layout(
    title=f'Matched Time and Scaling Factor',
    xaxis_title='t',
    yaxis_title='Value',
    height=500,
    width=700,
)
fig.show()

In [ ]:
def rf_trainer(rectified_flow, label = "loss", batch_size = 1024):
    model = rectified_flow.velocity_field
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

    losses = []
    for step in range(5000):
        optimizer.zero_grad()
        x_0 = pi_0.sample([batch_size]).to(device)
        x_1 = pi_1.sample([batch_size]).to(device)

        loss = rectified_flow.get_loss(x_0, x_1)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        if step % 1000 == 0:
            print(f"Epoch {step}, Loss: {loss.item()}")

    plt.plot(losses, label=label)
    plt.legend()

from rectified_flow.models.toy_mlp import MLPVelocity

set_seed(0)
straight_rf = RectifiedFlow(
    data_shape=(2,),
    velocity_field=MLPVelocity(2, hidden_sizes = [128, 128, 128]).to(device),
    interp=straight_interp,
    source_distribution=pi_0,
    device=device,
)

set_seed(0)
spherical_rf = RectifiedFlow(
    data_shape=(2,),
	velocity_field=MLPVelocity(2, hidden_sizes = [128, 128, 128]).to(device),
	interp=spherical_interp,
	source_distribution=pi_0,
	device=device,
)

set_seed(0)
rf_trainer(rectified_flow=straight_rf, label="straight interp")

set_seed(0)
rf_trainer(rectified_flow=spherical_rf, label="spherical interp")

In [ ]:
from rectified_flow.samplers import EulerSampler

euler_sampler_straight = EulerSampler(straight_rf, num_steps=100)
euler_sampler_straight.sample_loop(seed=0, num_samples=500)

euler_sampler_spherical = EulerSampler(spherical_rf, num_steps=100)
euler_sampler_spherical.sample_loop(seed=0, num_samples=500)

visualize_2d_trajectories_plotly(
    trajectories_dict={
        "1rf straight": euler_sampler_straight.trajectories,
        "1rf spherical": euler_sampler_spherical.trajectories,
	},
    D1_gt_samples=D1[:2000],
    num_trajectories=100,
    title="Euler Sampler Visualization",
)

Now, let’s take a pretrained straight rectified flow and transform it into a curved trajectory. In practice, we can leverage the existing velocity predictions from the straight path and reapply them to a new, curved rectified flow. Given the current postion $(z'_t,t)$ on $\{Z'_t\}$, we aim to use the pretrained velocity and convert it accordingly.

1. **Mapping to the New Trajectory**:  
   First, we find the corresponding position on the straight trajectory $\{Z_t\}$ for any given point $Z'_t$ on the curved trajectory $\{Z'_t\}$. This ensures we can reuse the pre-trained velocity field, which is defined along the straight path.

2. **Velocity Predictions**:  
   With the mapping established, we can now use the trained velocity model on $\{Z_t\}$ to obtain predictions $\hat{X}_0$ and $\hat{X}_1$. These predictions are crucial for ensuring that our curved interpolation still respects the underlying distributions.

3. **Updating the Trajectory**:  
   Finally, we advance the state along the curved trajectory using the updated interpolation $\mathcal{I}(\hat{X}_0, \hat{X}_1)$. This step integrates our predictions and ensures the resulting flow truly follows the curved path we’ve chosen.

By following these steps, we effectively "re-route" a rectified flow—originally trained on a straight interpolation—onto a different curve, all without needing to retrain the underlying model.

In [7]:
target_interp = AffineInterp("spherical")

convert_spherical_rf = AffineInterpConverter(straight_rf, target_interp).transform_rectified_flow()

We’ve successfully converted the pretrained straight rectified flow into a spherical one.

Next, let’s perform sampling using an Euler sampler on the two trajectories.

Let $\frac{\mathrm d}{\mathrm dt} Z_t = v_t(Z_t)$ be the rectified flow of $\{X_t\}$, which is initialized with $Z_0 = X_0$. Then $Z'_t = \phi_t(Z_{\tau_t})$ is the rectified flow of $\{X'_t\}$ with a specific initialization

$$
\frac{\mathrm d}{\mathrm dt} Z'_t = v'_t(Z'_t), \quad \forall t \in [0, 1], \quad \text{and} \quad Z'_0 = \phi_0(Z_{\tau_0}).
$$

In other words, the transformed trajectories lead to the same destinations! Despite following entirely different trajectories, their final generated samples are exactly identical.

By varying the `num_steps` in the following cell, we notice that as the number of steps increases, the Mean Squared Error (MSE) between the generated $ Z_1 $ and $ Z_1' $ decreases, confirming the consistency of the rectified coupling.

In [ ]:
# Try different num_steps, e.g. [5, 10, 50, 100, 500]
num_samples = 500
num_steps = 10

euler_sampler_straight = EulerSampler(straight_rf, num_steps=num_steps)
euler_sampler_straight.sample_loop(seed=0, num_samples=num_samples)

euler_sampler_converted_spherical = EulerSampler(convert_spherical_rf, num_steps=num_steps, num_samples=num_samples)
euler_sampler_converted_spherical.sample_loop(seed=0)

mse = torch.mean((euler_sampler_straight.trajectories[-1] - euler_sampler_converted_spherical.trajectories[-1])**2)
print(mse)

# zoom in to see they are really close
visualize_2d_trajectories_plotly(
	trajectories_dict={
        "straight rf": euler_sampler_straight.trajectories, 
        "straight to spherical rf": euler_sampler_converted_spherical.trajectories
    },
	D1_gt_samples=D1[:2000],
	num_trajectories=100,
	title=f"Straight Converted to Spherical   v.s.   Original Straight RF",
)

Obeserve that the converted spherical rf is nearly the same as the spherical rf

The only difference lies in the velocity paramitrization, which is not a big deal

In [ ]:
# Try different num_steps, e.g. [5, 10, 50, 100, 500]
num_samples = 500
num_steps = 100
euler_sampler_converted_spherical = EulerSampler(convert_spherical_rf, num_steps=num_steps, num_samples=num_samples)
euler_sampler_converted_spherical.sample_loop(seed=0)

euler_sampler_spherical = EulerSampler(spherical_rf, num_steps=num_steps)
euler_sampler_spherical.sample_loop(seed=0, num_samples=num_samples)

mse = torch.mean((euler_sampler_spherical.trajectories[-1] - euler_sampler_converted_spherical.trajectories[-1])**2)
print(mse)

# zoom in to see they are really close
visualize_2d_trajectories_plotly(
	trajectories_dict={
        "spherical rf": euler_sampler_spherical.trajectories, 
        "converted spherical rf": euler_sampler_converted_spherical.trajectories
    },
	D1_gt_samples=D1[:2000],
	num_trajectories=100,
	title=f"Converte Straight to Spherical RF v.s. Spherical RF",
)

Now we re-parametrize the straight rf, and learn it with spherical interpolation.

In [ ]:
def reparametrized_velocity_field(x, t):
    tau, omega = AffineInterpConverter.match_time_and_scale(AffineInterp("straight"), AffineInterp("spherical"), t)
    omega = omega.unsqueeze(-1)
    v_tau = spherical_rf_reparametrized_model(omega * x, tau)
    v = torch.pi / 2.0 * omega * (
        v_tau + (torch.cos(torch.pi * t / 2.) - torch.sin(torch.pi * t / 2.)).unsqueeze(-1) * x
    )
    return v
    
set_seed(0)
spherical_rf_reparametrized_model = MLPVelocity(2, hidden_sizes = [128, 128, 128]).to(device)

spherical_rf_reparametrized = RectifiedFlow(
    data_shape=(2,),
    velocity_field=reparametrized_velocity_field,
    interp=spherical_interp,
    source_distribution=pi_0,
    device=device,
)

batch_size = 1024

set_seed(0)
optimizer = torch.optim.Adam(spherical_rf_reparametrized_model.parameters(), lr=1e-2)

losses = []
for step in range(5000):
    optimizer.zero_grad()
    x_0 = pi_0.sample([batch_size]).to(device)
    x_1 = pi_1.sample([batch_size]).to(device)

    loss = spherical_rf_reparametrized.get_loss(x_0, x_1) / torch.pi * 2
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

    if step % 1000 == 0:
        print(f"Epoch {step}, Loss: {loss.item()}")

plt.plot(losses, label="reparametrized loss")
plt.legend()

In [ ]:
# Try different num_steps, e.g. [5, 10, 50, 100, 500]
num_samples = 500
num_steps = 100
euler_sampler_converted_spherical = EulerSampler(convert_spherical_rf, num_steps=num_steps, num_samples=num_samples)
euler_sampler_converted_spherical.sample_loop(seed=0)

euler_sampler_spherical = EulerSampler(spherical_rf_reparametrized, num_steps=num_steps)
euler_sampler_spherical.sample_loop(seed=0, num_samples=num_samples)

mse = torch.mean((euler_sampler_spherical.trajectories[-1] - euler_sampler_converted_spherical.trajectories[-1])**2)
print(mse)

# zoom in to see they are really close
visualize_2d_trajectories_plotly(
	trajectories_dict={
        "reparam spherical rf": euler_sampler_spherical.trajectories, 
        "straight to spherical rf": euler_sampler_converted_spherical.trajectories
    },
	D1_gt_samples=D1[:2000],
	num_trajectories=100,
	title=f"Re-parametrized Straight RF  v.s.  Converted Straight to Spherical RF",
)